In [1]:
import requests
from web3 import Web3
import pandas as pd

import libs.common.utils as utils
import libs.aave.api as aave_api
import libs.common.payload as payload
import config.config as cfg

In [3]:
w3 = Web3(Web3.HTTPProvider(cfg.config["http_url"]))

In [4]:
utils.get_function_hex("balanceOf(address)") == "0x70a08231"

True

In [5]:
utils.get_log_topic("Transfer(address,address,uint256)") == "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"

True

In [7]:
val = payload.erc20_decimals_payload(0, "0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2")
resp = requests.post(cfg.config["http_url"], json=val)
dec = int(resp.json()['result'],16)
assert dec == 18

True

In [8]:
val = payload.eth_block_payload(0, hex(18471066))
resp = requests.post(cfg.config["http_url"], json=val)
timestamp = resp.json()['result']['timestamp']
assert timestamp == "0x65412017"

True

In [12]:
wallet = "0x756F45E3FA69347A9A973A725E3C98bC4db0b5a0"
block_number = 0
val = payload.eth_balance_payload(0, wallet, hex(block_number))
resp = requests.post(cfg.config["http_url"], json=val)
assert int(resp.json()['result'],16)/1e18 == 200

In [13]:
val = payload.eth_logs_by_block_payload(0, hex(18471066), hex(18471066))
resp = requests.post(cfg.config["http_url"], json=val)
assert len(resp.json()['result']) == 580

In [14]:
# log is identified by address and topics
df_logs = pd.DataFrame(resp.json()['result'])
df_logs["timestamp"] = timestamp
log_sig = "Transfer(index_topic_1 address from, index_topic_2 address to, uint256 value)"
log_hash, log_name, log_topics, log_data = utils.parse_log_signature(log_sig)
df_logs = utils.parse_log_data(df_logs.loc[df_logs["topics"].str[0] == log_hash].copy(), log_topics, log_data, log_name = log_name)
assert df_logs.shape[0] == 385

In [ ]:
val = payload.eth_logs_by_block_address_payload(0, hex(18470000), hex(18471066), "0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599")
resp = requests.post(cfg.config["http_url"], json=val)
assert len(resp.json()['result']) == 789